In [1]:
import folium
import math
import random
import h5py
import folium 
import random
import numpy as np
import matplotlib.pyplot as plt
import pickle
import torch
import numpy as np
import os, sys, pickle

In [2]:
# load model and data
# get the prediction result and the ground truth and the real trajectory
# plot all of them on a map

In [4]:
# check the current working directory
os.chdir('/home/yangshuaiyu6791/RepresentAttack/inversion_sePoint/')

from modules.models import sPoints_net, ePoints_net, se_net
from DataLoader.dataloader import TrajDataset

# load pth
model = se_net(128,512)
params1 = torch.load("./models/d0_002_train_sePoints_neutraj_128_512.pth",
                    map_location=torch.device('cpu'))
model.load_state_dict(params1)
model.eval()

# load data
with open("./data/neutraj/128/train/train_data", "rb") as f:
    tmp_embs = pickle.load(f)
    train_embs = tmp_embs['embs']
with open("./data/neutraj/128/train/train_grid", "rb") as f:
    train_grids = pickle.load(f)

with open("./data/neutraj/128/val/val_data", "rb") as f:
    tmp_embs = pickle.load(f)
    val_embs = tmp_embs['embs']
with open("./data/neutraj/128/val/val_grid", "rb") as f:
    val_grids = pickle.load(f)

with open("./rdata/gps_seqs/traingps", "rb") as f:
    train_gps = pickle.load(f)
    train_gps = train_gps[0:60000]
with open("./rdata/gps_seqs/valgps", "rb") as f:
    val_gps = pickle.load(f)

In [13]:
# get the prediction result and the ground truth and the real trajectory
# plot all of them on a map

choice = random.randint(0, len(val_embs))
print(choice)

# get the prediction result and the grid
val_emb = torch.from_numpy(val_embs[choice]).float()
pred_sePoints = model(val_emb)
pred_sePoints = pred_sePoints.detach().numpy()
pred_sPoint = np.array([pred_sePoints[0], pred_sePoints[1]])
pred_ePoint = np.array([pred_sePoints[2], pred_sePoints[3]])
val_grid = val_grids[choice]; grid_lenth = len(val_grid)
grid_sPoint = val_grid[0]; grid_ePoint = val_grid[grid_lenth-1]

print(pred_sPoint); print(pred_ePoint)
print(grid_sPoint); print(grid_ePoint)

7067


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [11]:
# map grid to gps
# cellnum: [57,35,1995];  the true cell_index = x + y* cellnum[0]
# x,y need to round float to int

porto_lat_range = [40.953673,41.307945]
porto_lon_range = [-8.735152,-8.156309]
delta = 0.002

def grid2gps(grid):    # x -> lon, y -> lat
    grid_x = grid[0]; grid_y = grid[1]
    gps_x = porto_lon_range[0] + grid_x * delta
    gps_y = porto_lat_range[0] + grid_y * delta
    return [gps_x, gps_y]    # [lon, lat]


In [ ]:
pred_sPoint = pred_sPoint; pred_ePoint = pred_ePoint
grid_sPoint = np.array(grid_sPoint); grid_ePoint = np.array(grid_ePoint)
pred_sPoint = [round(pred_sPoint[0]), round(pred_sPoint[1])]
pred_ePoint = [round(pred_ePoint[0]), round(pred_ePoint[1])]
grid_sPoint = [round(grid_sPoint[0]), round(grid_sPoint[1])]
grid_ePoint = [round(grid_ePoint[0]), round(grid_ePoint[1])]
print(pred_sPoint); print(pred_ePoint); print(grid_sPoint); print(grid_ePoint)

gps_pred_sPoint = grid2gps(pred_sPoint); gps_pred_ePoint = grid2gps(pred_ePoint)
gps_grid_sPoint = grid2gps(grid_sPoint); gps_grid_ePoint = grid2gps(grid_ePoint)
print(gps_pred_sPoint); print(gps_pred_ePoint); print(gps_grid_sPoint); print(gps_grid_ePoint)


[52, 87]
[57, 85]
[59, 97]
[64, 97]
[-8.631152, 41.127673]
[-8.621151999999999, 41.123673000000004]
[-8.617151999999999, 41.147673000000005]
[-8.607152, 41.147673000000005]


In [ ]:
my_map = folium.Map(location=[40.953673,-8.735152], zoom_start=15)
folium.CircleMarker(location=[gps_pred_sPoint[1],gps_pred_sPoint[0]], radius=5, color='red', fill=True).add_to(my_map)
folium.CircleMarker(location=[gps_pred_ePoint[1],gps_pred_ePoint[0]], radius=5, color='red', fill=True).add_to(my_map)
folium.CircleMarker(location=[gps_grid_sPoint[1],gps_grid_sPoint[0]], radius=5, color='blue', fill=True).add_to(my_map)
folium.CircleMarker(location=[gps_grid_ePoint[1],gps_grid_ePoint[0]], radius=5, color='blue', fill=True).add_to(my_map)

# plot the real trajectory
gps_train = train_gps[choice][:,[1,0]]
folium.PolyLine(gps_train, color="purple", weight=15, opacity=0.6).add_to(my_map)

my_map.save("./results/plot1.html")
my_map